In [1]:
from sklearn.metrics import roc_auc_score, precision_recall_curve
from sklearn.metrics import auc as calculate_auc
from sklearn.metrics import mean_squared_error
from sklearn.metrics import accuracy_score
from tqdm import tqdm
from sklearn.utils import shuffle 
from joblib import load, dump
import numpy as np
import pandas as pd
import os,sys,json
from argparse import ArgumentParser
from scipy.stats.stats import pearsonr
import time
import xgboost

In [2]:
def rmse(y_true, y_pred):
	mse = mean_squared_error(y_true, y_pred)
	rmse = np.sqrt(mse)  
	return rmse

# Load y

In [3]:
df=pd.read_csv('CB2_Chembl_R6227.csv')

In [4]:
df.head()

,smiles,pValue
0,CCCCCOc1c(OC)ccc2cc(C(=O)NCCc3ccncc3)c(=O)[nH]c12,11.00
1,CCCCCc1cc2cc(C(=O)OC)c(=O)[nH]c2c(O)c1OC,10.85
2,CCCCCOc1c(OC)ccc2cc(C(=O)NCCc3ccc(F)cc3)c(=O)[...,10.72
3,CCOc1c(OC)ccc2cc(C(=O)NCCc3ccncc3)c(=O)[nH]c12,10.70
4,CCCCCOc1c(OC)ccc2cc(C(=O)NCCc3ccc(F)cc3)c(=O)n...,10.68


In [5]:
Y0 = df['PActivity'].astype('float').values

In [6]:
Y0

array([11.  , 10.85, 10.72, ...,  4.  ,  4.  ,  4.  ])

In [8]:
len(Y0)

6227

# Load feats, i.e. X

In [9]:
X0=pd.read_csv('AvalonFP_AtomPairFP_RDkitFP_MorganFP.csv')

In [10]:
X0.head()

,AvalonFP0,AvalonFP1,AvalonFP2,AvalonFP3,AvalonFP4,AvalonFP5,AvalonFP6,AvalonFP7,AvalonFP8,AvalonFP9,...,MorganFP2038,MorganFP2039,MorganFP2040,MorganFP2041,MorganFP2042,MorganFP2043,MorganFP2044,MorganFP2045,MorganFP2046,MorganFP2047
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [11]:
len(X0)

6227

# splitting data in random mode, using seed=1
# i.e. X0,Y0 will be splitted into  train, valid, test dataset

In [12]:
def random_split(df, random_state, split_size = [0.8, 0.1, 0.1]):
	base_indices = np.arange(len(df)) 
	base_indices = shuffle(base_indices, random_state = random_state) 
	nb_test = int(len(base_indices) * split_size[2]) 
	nb_val = int(len(base_indices) * split_size[1]) 
	test_idx = base_indices[0:nb_test] 
	valid_idx = base_indices[(nb_test):(nb_test+nb_val)] 
	train_idx = base_indices[(nb_test+nb_val):len(base_indices)] 
	print(len(train_idx), len(valid_idx), len(test_idx)) 
	return train_idx, valid_idx, test_idx 

In [15]:
seed = 1
X0=X0.values

In [16]:
train_idx, valid_idx, test_idx = random_split(df,random_state=seed)
train_idx = [i for i in train_idx if i < len(df)]
valid_idx = [i for i in valid_idx if i < len(df)]	
test_idx = [i for i in test_idx if i < len(df)]	
print(len(train_idx), len(valid_idx), len(test_idx)) 
X = X0[train_idx]; y = Y0[train_idx]
X_valid = X0[valid_idx];y_valid = Y0[valid_idx]
X_test = X0[test_idx]; y_test = Y0[test_idx] 

4983 622 622
4983 622 622


# loading optimized parameters

In [17]:
import json
with open('params_regress','r') as f:
    best_param=json.load(f)

In [18]:
best_param

{'params': {'base_score': 0.5,
  'booster': 'gbtree',
  'colsample_bylevel': 1,
  'colsample_bynode': 1,
  'colsample_bytree': 0.3,
  'gamma': 0.02,
  'gpu_id': 0,
  'importance_type': 'gain',
  'learning_rate': 0.05,
  'max_delta_step': 0,
  'max_depth': 6,
  'min_child_weight': 4,
  'missing': None,
  'n_estimators': 10000,
  'n_jobs': 1,
  'nthread': None,
  'objective': 'reg:squarederror',
  'random_state': 0,
  'reg_alpha': 4.0,
  'reg_lambda': 1.2,
  'scale_pos_weight': 1,
  'seed': 123,
  'silent': None,
  'subsample': 0.6,
  'tree_method': 'gpu_hist',
  'verbosity': 1},
 'results': {'fp_types': 'AvalonFP_AtomPairFP_RDkitFP_MorganFP',
  'model': 'xgb',
  'test_r2': 0.7018138890790199,
  'test_rmse': 0.6490377959563551,
  'time': 129.38433146476746,
  'train_r2': 0.9847055980981125,
  'train_rmse': 0.14417539136408952,
  'valid_r2': 0.657361636795885,
  'valid_rmse': 0.6536675629974252}}

In [19]:
use_param=best_param['params']

In [20]:
use_param

{'base_score': 0.5,
 'booster': 'gbtree',
 'colsample_bylevel': 1,
 'colsample_bynode': 1,
 'colsample_bytree': 0.3,
 'gamma': 0.02,
 'gpu_id': 0,
 'importance_type': 'gain',
 'learning_rate': 0.05,
 'max_delta_step': 0,
 'max_depth': 6,
 'min_child_weight': 4,
 'missing': None,
 'n_estimators': 10000,
 'n_jobs': 1,
 'nthread': None,
 'objective': 'reg:squarederror',
 'random_state': 0,
 'reg_alpha': 4.0,
 'reg_lambda': 1.2,
 'scale_pos_weight': 1,
 'seed': 123,
 'silent': None,
 'subsample': 0.6,
 'tree_method': 'gpu_hist',
 'verbosity': 1}

# define xgboost regression model

In [21]:
clf=xgboost.XGBRegressor(**use_param)
#clf=xgboost.XGBRegressor(**best_param['params'])

In [22]:
print('clf = ',clf)

clf =  XGBRegressor(colsample_bytree=0.3, gamma=0.02, gpu_id=0, learning_rate=0.05,
             max_depth=6, min_child_weight=4, n_estimators=10000,
             objective='reg:squarederror', reg_alpha=4.0, reg_lambda=1.2,
             seed=123, subsample=0.6, tree_method='gpu_hist')


# training xgb using training dataset

In [23]:
time1=time.time()
model=clf.fit(X, y)
time2=time.time()
time_fit=time2-time1
print(f"fit time is: {time_fit}")

fit time is: 291.88478422164917


# evaluating the performance by valid and test dataset

In [25]:
valid_r2 = pearsonr(y_valid, clf.predict(X_valid))[0]**2
valid_rmse = rmse(y_valid, clf.predict(X_valid))
test_r2 = pearsonr(y_test, clf.predict(X_test))[0]**2
test_rmse = rmse(y_test, clf.predict(X_test))

In [28]:
results = {"seed":seed, 'valid_rmse':valid_rmse,
'valid_r2':valid_r2,"test_rmse":test_rmse, "test_r2": test_r2,"time":time_fit}
print('results = ',results)

results =  {'seed': 1, 'valid_rmse': 0.6619354805986309, 'valid_r2': 0.662065566393008, 'test_rmse': 0.6892037357815582, 'test_r2': 0.6695890564861264, 'time': 291.88478422164917}


# change random seed to 8 to split the X0 and Y0, rerun the training and evaluation

In [30]:
seed = 8

train_idx, valid_idx, test_idx = random_split(df,random_state=seed)
train_idx = [i for i in train_idx if i < len(df)]
valid_idx = [i for i in valid_idx if i < len(df)]	
test_idx = [i for i in test_idx if i < len(df)]	
print(len(train_idx), len(valid_idx), len(test_idx)) 
X = X0[train_idx]; y = Y0[train_idx]
X_valid = X0[valid_idx];y_valid = Y0[valid_idx]
X_test = X0[test_idx]; y_test = Y0[test_idx] 

4983 622 622
4983 622 622


# re-define xgb model

In [32]:
clf=''
clf=xgboost.XGBRegressor(**use_param)

In [33]:
time1=time.time()
model=clf.fit(X, y)
time2=time.time()
time_fit=time2-time1
print(f"fit time is: {time_fit}")
valid_r2 = pearsonr(y_valid, clf.predict(X_valid))[0]**2
valid_rmse = rmse(y_valid, clf.predict(X_valid))
test_r2 = pearsonr(y_test, clf.predict(X_test))[0]**2
test_rmse = rmse(y_test, clf.predict(X_test))
results = {"seed":seed, 'valid_rmse':valid_rmse,
'valid_r2':valid_r2,"test_rmse":test_rmse, "test_r2": test_r2,"time":time_fit}
print('results = ',results)

fit time is: 285.77695178985596
results =  {'seed': 8, 'valid_rmse': 0.6671932464346332, 'valid_r2': 0.6437878699997245, 'test_rmse': 0.6635918275212866, 'test_r2': 0.6883650373171917, 'time': 285.77695178985596}
